In [7]:
import numpy as np
import pickle as pkl
import os
import argparse
import pandas as pd
import glob

import preprocess.vocabulary as mv
import preprocess.data_preparation as pdp
import configuration.config_default as cfgd
import utils.log as ul
import utils.file as uf
import preprocess.property_change_encoder as pce
from pathlib import Path
from pandarallel import pandarallel
from functools import partial
import numpy as np
import copy
pandarallel.initialize(nb_workers=64)

def gen_tokens(smiles_list):
    property_condition=[]
    # add property name before property change; save to file
    tokenizer = mv.SMILESTokenizer()
    vocabulary = mv.create_vocabulary(smiles_list, tokenizer=tokenizer, property_condition=property_condition)
    tokens = vocabulary.tokens()
    return tokens

def token_inVocab(row, trainTokens):
    smiList=[row['constantSMILES'],row['fromVarSMILES'],row['toVarSMILES']]
    tokens=gen_tokens(smiList)
    if set(tokens).issubset(trainTokens):
        return 1
    else:
        return 0
    


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


##   Filter the SMILES containing token that not in train vocabulary
- 



In [ ]:
vocab_path="/public/home/zhangjie/Projects/MMP/pot_clm/CLModel_v2/PretrainWork/ChEMBL32_Data"
csvFile_path="/public/home/zhangjie/Projects/MMP/pot_clm/CLModel_v2/FinetuningData/AIXB-3/**"

## load train vocabulary
with open(os.path.join(vocab_path, 'vocab.pkl'), "rb") as input_file:
    vocab = pkl.load(input_file)
    trainTokens=vocab.tokens()
token_inVocab_p=partial(token_inVocab, trainTokens=trainTokens)

csvFiles=glob.glob(f"{csvFile_path}/*_MMP.csv",recursive=True)   
for immpFile in csvFiles: 
    print(f"Working on {immpFile}")
    df_mmp=pd.read_csv(immpFile)
    print(f"Orginal size: {len(df_mmp)}")
    df_mmp['tokenFilter']=df_mmp.parallel_apply(lambda x: token_inVocab_p(x), axis=1)
    df_mmp=df_mmp[df_mmp['tokenFilter']==1]
    print(f"Final size: {len(df_mmp)}")
    df_mmp.to_csv(immpFile.replace('.csv', '_filtered.csv'), index=None)
    
    ''' preprocess for training '''
    os.system(f"python preprocess.py -i  {immpFile.replace('.csv', '_filtered.csv')}  -d 1")
